In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('train.csv')
final = pd.read_csv('test.csv')

In [3]:
df.rename(columns= {col: col.lower().replace(' ','_') for col in df.columns}, inplace=True)

In [4]:
# alley              140 non-null object -- drop
# lot_frontage       1721 non-null float64 -- drop

# mas_vnr_type       2029 non-null object 
# mas_vnr_area       2029 non-null float64 -- has or doesnt have

# bsmt_qual          1996 non-null object
# bsmt_cond          1996 non-null object
# bsmt_exposure      1993 non-null object
# bsmtfin_type_1     1996 non-null object
# bsmtfin_sf_1       2050 non-null float64
# bsmtfin_type_2     1995 non-null object
# bsmtfin_sf_2       2050 non-null float64
# bsmt_unf_sf        2050 non-null float64
# total_bsmt_sf      2050 non-null float64 -- change to basement or no basement
# bsmt_full_bath     2049 non-null float64
# bsmt_half_bath     2049 non-null float64

# fireplace_qu       1051 non-null object -- drop, will use 'fireplaces' and set to yes or no

# garage_type        1938 non-null object -- change to garage or no garage
# garage_yr_blt      1937 non-null float64 ------check misc_feature for Gar2 --> they are all caught in garage_cars
# garage_finish      1937 non-null object
# garage_cars        2050 non-null float64 -- use this value for if it has garage ... if > 0
# garage_area        2050 non-null float64
# garage_qual        1937 non-null object
# garage_cond        1937 non-null object

# pool_qc            9 non-null object -- drop, will use 'pool_area > 0'
# fence              400 non-null object -- change to has fence or no fence
# misc_feature       65 non-null object -- drop

# -----still need to do-----

# fence              400 non-null object -- change to has fence or no fence
# pool_area > 0
# garage_cars        2050 non-null float64 -- use this value for if it has garage ... if > 0
# 'fireplaces' and set to yes or no
# total_bsmt_sf      2050 non-null float64 -- change to basement or no basement
# mas_vnr_area       2029 non-null float64 -- has or doesnt have

In [5]:
df.drop([
    'alley', 'lot_frontage', 'mas_vnr_type', 'fireplace_qu', 'pool_qc', 'misc_feature',
    'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2',
    'bsmtfin_sf_2', 'bsmt_unf_sf', 'bsmt_full_bath', 'bsmt_half_bath',
    'garage_type', 'garage_yr_blt', 'garage_finish', 'garage_area', 'garage_qual', 'garage_cond',
], axis=1, inplace=True)

In [6]:
df['fence'] = df['fence'].notnull().astype('int')

In [7]:
df.head(10)

,id,pid,ms_subclass,ms_zoning,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,109,533352170,60,RL,13517,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2010,WD,Normal,130500
1,544,531379050,60,RL,11492,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,4,2009,WD,Normal,220000
2,153,535304180,20,RL,7922,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,1,2010,WD,Abnorml,109000
3,318,916386060,60,RL,9802,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2010,WD,Normal,174000
4,255,906425045,50,RL,14235,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,138500
5,138,535126040,20,RL,16492,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,6,2010,WD,Normal,190000
6,2827,908186070,180,RM,3675,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2006,New,Partial,140000
7,145,535154050,20,RL,12160,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,5,2010,COD,Abnorml,142000
8,1942,535353130,20,RL,15783,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,1,400,6,2007,WD,Normal,112500
9,1956,535426130,60,RL,11606,Pave,IR1,HLS,AllPub,Inside,...,0,0,0,0,0,9,2007,WD,Family,135000


In [8]:
features_replace = ['pool_area' , 'garage_cars', 'fireplaces', 'total_bsmt_sf', 'mas_vnr_area']

def does_does_not(cell):
    if cell >= 1:
        return 1
    else:
        return 0
    
for i in features_replace:
    df[i] = df[i].map(does_does_not)

In [9]:
df.rename(columns={
    'pool_area': 'has_pool',
    'garage_cars': 'has_garage', 
    'fireplaces': 'has_fireplace',
    'total_bsmt_sf': 'has_bsmt',
    'mas_vnr_area': 'has_mas_vnr',
    'fence': 'has_fence'
}, inplace=True)

In [10]:
df.head(15)

,id,pid,ms_subclass,ms_zoning,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,3ssn_porch,screen_porch,has_pool,has_fence,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,109,533352170,60,RL,13517,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2010,WD,Normal,130500
1,544,531379050,60,RL,11492,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,4,2009,WD,Normal,220000
2,153,535304180,20,RL,7922,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,1,2010,WD,Abnorml,109000
3,318,916386060,60,RL,9802,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2010,WD,Normal,174000
4,255,906425045,50,RL,14235,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,138500
5,138,535126040,20,RL,16492,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,6,2010,WD,Normal,190000
6,2827,908186070,180,RM,3675,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2006,New,Partial,140000
7,145,535154050,20,RL,12160,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,5,2010,COD,Abnorml,142000
8,1942,535353130,20,RL,15783,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,1,400,6,2007,WD,Normal,112500
9,1956,535426130,60,RL,11606,Pave,IR1,HLS,AllPub,Inside,...,0,0,0,0,0,9,2007,WD,Family,135000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 60 columns):
id                 2051 non-null int64
pid                2051 non-null int64
ms_subclass        2051 non-null int64
ms_zoning          2051 non-null object
lot_area           2051 non-null int64
street             2051 non-null object
lot_shape          2051 non-null object
land_contour       2051 non-null object
utilities          2051 non-null object
lot_config         2051 non-null object
land_slope         2051 non-null object
neighborhood       2051 non-null object
condition_1        2051 non-null object
condition_2        2051 non-null object
bldg_type          2051 non-null object
house_style        2051 non-null object
overall_qual       2051 non-null int64
overall_cond       2051 non-null int64
year_built         2051 non-null int64
year_remod/add     2051 non-null int64
roof_style         2051 non-null object
roof_matl          2051 non-null object
exterior_1st       20

In [ ]:
# pipline for cleaning so far

In [ ]:
pipe = Pipeline